# VAZHI v0.4 Training

**Goal**: Train Tamil LLM with complete 6-pack training data

## All Packs Included:
- 🪷 **கலாச்சாரம்** (Culture) - Thirukkural, Siddhars, temples, festivals
- 📚 **கல்வி** (Education) - Scholarships, exams, admissions
- 🛡️ **பாதுகாப்பு** (Security) - Cyber safety, scam prevention
- ⚖️ **சட்டம்** (Legal) - RTI, consumer rights, legal procedures
- 🏛️ **அரசு** (Government) - Government schemes & services
- 🏥 **சுகாதாரம்** (Healthcare) - Health, Siddha medicine

## Training Data:
- Training: 2,732 samples
- Validation: 301 samples
- Total: 3,033 samples

## Cell 1: Install Dependencies

In [ ]:
# Install Unsloth and dependencies
!pip install unsloth
!pip install --no-deps trl peft accelerate bitsandbytes
!pip install datasets

## Cell 2: Upload Training Data

Upload these files from `data/v04/training/`:
- `vazhi_v04_train.json` (2,732 samples)
- `vazhi_v04_val.json` (301 samples)

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload vazhi_train_v02.json and vazhi_val_v02.json

In [ ]:
# Verify uploaded files
import os
print("Uploaded files:")
for f in os.listdir('.'):
    if f.endswith('.json'):
        print(f"  - {f} ({os.path.getsize(f) / 1024:.1f} KB)")

## Cell 3: Load Base Model with Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

# Model configuration
MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"
MAX_SEQ_LENGTH = 2048

# Load model in 4-bit for memory efficiency
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,  # Auto-detect
    load_in_4bit=True,
)

print("Model loaded successfully!")
print(f"Model parameters: {model.num_parameters():,}")

## Cell 4: Add LoRA Adapters

In [ ]:
# Configure LoRA for Tamil fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

print("LoRA adapters added!")
print(f"Trainable parameters: {model.num_parameters(only_trainable=True):,}")

## Cell 5: Define Tamil Chat Template

In [ ]:
# Skip this cell - formatting is done in Cell 6
pass

## Cell 6: Load Training Data

In [ ]:
from datasets import Dataset
import json

# Tamil system prompt for VAZHI
SYSTEM_PROMPT = """நீங்கள் VAZHI (வழி), தமிழ் மக்களுக்கான AI உதவியாளர். தமிழில் தெளிவாகவும் உதவியாகவும் பதிலளியுங்கள். You can respond in Tamil, Tanglish, or English based on how the user asks."""

def format_example(example):
    """Format a training example into Qwen chat template."""
    return f"""<|im_start|>system
{SYSTEM_PROMPT}<|im_end|>
<|im_start|>user
{example['instruction']}<|im_end|>
<|im_start|>assistant
{example['output']}<|im_end|>"""

# Load v0.4 training data
with open("vazhi_v04_train.json", "r") as f:
    train_data = json.load(f)

with open("vazhi_v04_val.json", "r") as f:
    val_data = json.load(f)

# Pre-format all examples (avoid dataset.map stalling)
print("Formatting training data...")
train_formatted = [{"text": format_example(ex)} for ex in train_data]
print("Formatting validation data...")
val_formatted = [{"text": format_example(ex)} for ex in val_data]

print(f"\nTraining samples: {len(train_formatted)}")
print(f"Validation samples: {len(val_formatted)}")
print(f"Total: {len(train_formatted) + len(val_formatted)}")

# Convert to HuggingFace Dataset
train_dataset = Dataset.from_list(train_formatted)
val_dataset = Dataset.from_list(val_formatted)

print("\nSample formatted text:")
print(train_dataset[0]["text"][:500])

In [ ]:
# Check pack distribution (from original data before formatting)
from collections import Counter

pack_counts = Counter(item.get('pack', 'unknown') for item in train_data)
print("Pack distribution:")
for pack, count in sorted(pack_counts.items()):
    print(f"  {pack}: {count}")

## Cell 7: Configure Training Arguments

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

# Training configuration - optimized for T4 GPU
training_args = TrainingArguments(
    output_dir="./vazhi-v04-lora",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,  # Effective batch size = 8
    learning_rate=2e-4,
    warmup_steps=50,
    optim="adamw_8bit",
    weight_decay=0.01,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=10,
    save_steps=200,
    save_total_limit=3,
    eval_strategy="steps",
    eval_steps=200,
    seed=42,
    report_to="none",
)

print("Training arguments configured!")
print(f"Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")

## Cell 8: Initialize Trainer

In [ ]:
# Create the trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    packing=False,  # Disable packing to avoid issues
    args=training_args,
)

print("Trainer initialized!")
print(f"Training samples: {len(train_dataset)}")
print(f"Steps per epoch: {len(train_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)}")

## Cell 9: Train the Model

**Estimated time**: ~45-60 minutes on T4 GPU

In [ ]:
import time

print("="*50)
print("Starting VAZHI v0.4 Training")
print("="*50)

start_time = time.time()

# Train!
trainer.train()

end_time = time.time()
training_time = end_time - start_time

print("="*50)
print("Training Complete!")
print("="*50)
print(f"Total training time: {training_time:.2f} seconds ({training_time/60:.1f} minutes)")

## Cell 10: Save the Model

In [ ]:
# Save the LoRA adapter
model.save_pretrained("vazhi-v04-lora-final")
tokenizer.save_pretrained("vazhi-v04-lora-final")

print("Model saved to vazhi-v04-lora-final/")

# Download the model files
from google.colab import files
import shutil

shutil.make_archive("vazhi-v04-lora-final", 'zip', "vazhi-v04-lora-final")
files.download("vazhi-v04-lora-final.zip")
print("Model downloaded!")

## Cell 11: Define Inference Function (Improved)

In [ ]:
# Set model to inference mode
FastLanguageModel.for_inference(model)

def vazhi_chat(user_message: str, max_tokens: int = 512) -> str:
    """
    Chat with VAZHI v0.2 model.

    Args:
        user_message: User's question in Tamil/Tanglish/English
        max_tokens: Maximum response length (default 512, up from 256 in v0.1)

    Returns:
        Model's response
    """
    prompt = f"""<|im_start|>system
{SYSTEM_PROMPT}<|im_end|>
<|im_start|>user
{user_message}<|im_end|>
<|im_start|>assistant
"""

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=False)

    # Extract assistant response (improved parsing)
    if "<|im_start|>assistant" in response:
        response = response.split("<|im_start|>assistant")[-1]

    # Clean up markers
    response = response.replace("<|im_end|>", "").strip()
    response = response.replace("<|im_start|>", "").strip()

    return response

print("Inference function ready!")

## Cell 12: Comprehensive VAZHI v0.4 Test

In [ ]:
# Test questions across all 6 packs
test_questions = [
    # Culture - கலாச்சாரம்
    ("Culture", "திருக்குறளின் முதல் குறள் என்ன?"),
    ("Culture", "அகத்தியர் பற்றி சொல்லுங்கள்"),
    
    # Education - கல்வி
    ("Education", "12th படிச்சாச்சு, engineering-க்கு என்ன options?"),
    ("Education", "NEET exam-க்கு எப்படி prepare பண்றது?"),

    # Security - பாதுகாப்பு
    ("Security", "Scam message-ஐ எப்படி identify பண்றது?"),
    ("Security", "OTP யாருக்கும் share பண்ணக்கூடாது ஏன்?"),

    # Legal - சட்டம்
    ("Legal", "RTI என்றால் என்ன?"),
    ("Legal", "Consumer complaint எப்படி போடுவது?"),

    # Government - அரசு
    ("Government", "CMCHIS card எப்படி apply பண்றது?"),
    ("Government", "PM Kisan scheme என்றால் என்ன?"),

    # Healthcare - சுகாதாரம்
    ("Healthcare", "சித்த மருத்துவம் என்றால் என்ன?"),
    ("Healthcare", "Government hospital-ல் free treatment கிடைக்குமா?"),
]

print("="*60)
print("VAZHI v0.4 - Comprehensive Test (All 6 Packs)")
print("="*60)

for category, q in test_questions:
    print(f"\n[{category}] Q: {q}")
    print(f"A: {vazhi_chat(q)}")
    print("-"*50)

## Cell 13: Specific Thirukkural Validation

This is the critical test - v0.1 failed on Thirukkural accuracy.

In [ ]:
# Thirukkural accuracy test - these were WRONG in v0.1
thirukkural_tests = [
    {
        "question": "திருக்குறளின் முதல் குறள் என்ன?",
        "expected_contains": ["அகர முதல", "எழுத்தெல்லாம்", "ஆதி", "பகவன்"]
    },
    {
        "question": "கற்றதனால் ஆய பயனென்கொல் என்ற குறள் முழுமையாக சொல்லுங்கள்",
        "expected_contains": ["வாலறிவன்", "நற்றாள்", "தொழாஅர்"]
    },
    {
        "question": "பிறவிப் பெருங்கடல் என்ற குறள் என்ன?",
        "expected_contains": ["பிறவி", "கடல்", "நீந்து", "இறைவன்"]
    },
]

print("="*60)
print("Thirukkural Accuracy Validation")
print("="*60)

passed = 0
for test in thirukkural_tests:
    response = vazhi_chat(test["question"])
    matches = sum(1 for keyword in test["expected_contains"] if keyword in response)
    total = len(test["expected_contains"])
    status = "✅ PASS" if matches >= total * 0.75 else "❌ FAIL"

    if matches >= total * 0.75:
        passed += 1

    print(f"\nQ: {test['question']}")
    print(f"A: {response[:300]}..." if len(response) > 300 else f"A: {response}")
    print(f"Keywords found: {matches}/{total} {status}")

print(f"\n{'='*60}")
print(f"Thirukkural Tests: {passed}/{len(thirukkural_tests)} passed")
print("="*60)

## Cell 14: v0.1 vs v0.2 Comparison Summary

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════╗
║              VAZHI v0.4 Training Complete!                   ║
╠══════════════════════════════════════════════════════════════╣
║  Training Data:                                              ║
║  ┌─────────────────────┬───────────┐                         ║
║  │ Pack                │ Samples   │                         ║
║  ├─────────────────────┼───────────┤                         ║
║  │ Culture (கலாச்சாரம்)  │ ~500      │                         ║
║  │ Education (கல்வி)    │ ~460      │                         ║
║  │ Security (பாதுகாப்பு) │ ~450      │                         ║
║  │ Legal (சட்டம்)       │ ~440      │                         ║
║  │ Government (அரசு)   │ ~430      │                         ║
║  │ Healthcare (சுகாதாரம்)│ ~460      │                         ║
║  └─────────────────────┴───────────┘                         ║
║  Total: 3,033 samples (2,732 train + 301 val)               ║
║                                                              ║
║  Files saved:                                                ║
║  - vazhi-v04-lora-final.zip (~110MB)                        ║
║                                                              ║
║  Next steps:                                                 ║
║  1. Upload to HuggingFace: CryptoYogi/vazhi-lora            ║
║  2. Space will auto-load the adapter                        ║
║  3. Test Flutter app with new model                         ║
╚══════════════════════════════════════════════════════════════╝
""")

## Cell 15: (Optional) Upload to HuggingFace Hub

Run this cell to push your model directly to HuggingFace Hub.

In [ ]:
# Upload to HuggingFace Hub
from huggingface_hub import login, HfApi

# Login to HuggingFace (will prompt for token)
login()

# Push model to Hub
model.push_to_hub("CryptoYogi/vazhi-lora")
tokenizer.push_to_hub("CryptoYogi/vazhi-lora")

print("✅ Model uploaded to https://huggingface.co/CryptoYogi/vazhi-lora")